In [3]:
import pandas as pd
import numpy as np
import geopandas as gp

In [4]:
counties = gp.GeoDataFrame.from_file('tl_2020_us_county.shp')
list(counties)

#counties.STATE_NAME.unique()
#counties = counties[(counties.STATE_NAME != 'Hawaii') & (counties.STATE_NAME != 'Alaska')]
counties = counties[['STATEFP','COUNTYFP',"NAME",'NAMELSAD','geometry']]
counties.head()

,STATEFP,COUNTYFP,NAME,NAMELSAD,geometry
0,31,039,Cuming,Cuming County,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,Wahkiakum,Wahkiakum County,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,De Baca,De Baca County,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,Lancaster,Lancaster County,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,Nuckolls,Nuckolls County,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [5]:
counties.crs


<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [6]:
import rasterio
nlcd = 'NLCD_2004_Land_Cover_L48_20190424.img'
test = rasterio.open(nlcd)
test.meta

{'driver': 'HFA',
 'dtype': 'uint8',
 'nodata': None,
 'width': 161190,
 'height': 104424,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.0, 0.0, -2493045.0,
        0.0, -30.0, 3310005.0)}

In [7]:
counties = counties.to_crs(epsg="5070")

In [8]:
from rasterstats import raster_stats
from rasterstats import zonal_stats
import time

In [9]:
start_time = time.time()
county_stats = zonal_stats(counties, nlcd, stats="count majority minority unique",geojson_out=True,categorical=True)
elapsed_time = time.time() - start_time
elapsed_time

/Users/timdowling/anaconda3/lib/python3.6/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


958.3904087543488

In [10]:
county_stats[4]

{'id': '4',
 'type': 'Feature',
 'properties': {'COUNTYFP': '129',
  'NAME': 'Nuckolls',
  'NAMELSAD': 'Nuckolls County',
  'STATEFP': '31',
  11: 8031,
  21: 50970,
  22: 9410,
  23: 1287,
  24: 448,
  31: 222,
  41: 43868,
  43: 128,
  52: 3,
  71: 573957,
  81: 1730,
  82: 958297,
  90: 6623,
  95: 2096,
  'count': 1657070,
  'majority': 82.0,
  'minority': 52.0,
  'unique': 14},
 'geometry': {'type': 'Polygon',
  'coordinates': (((-192145.70411346076, 1899292.8464565892),
    (-192145.67194941937, 1899294.190570985),
    (-192141.07242357486, 1899405.1454871846),
    (-192140.2147249882, 1899440.9885166832),
    (-192140.1879219096, 1899442.1086112629),
    (-192140.1584385234, 1899443.3407152889),
    (-192140.12895513728, 1899444.572819311),
    (-192140.09947175125, 1899445.8049233283),
    (-192140.06730805774, 1899447.1490367895),
    (-192140.03514436432, 1899448.4931502463),
    (-192140.00030036346, 1899449.9492731425),
    (-192139.96545636276, 1899451.405396033),
    (-19

In [11]:
print(len(county_stats))

3234


In [12]:
all_counties = [x['properties'] for x in county_stats]

In [13]:
print(len(all_counties))

3234


In [14]:
non_zero = [d for d in all_counties if d['majority'] != 0] 

In [15]:
print(len(non_zero))

2471


In [16]:
#get landcover
open_water = []
for dic in non_zero:
    open_water.append(dic.get(11, None))

snow_ice = []
for dic in non_zero:
    snow_ice.append(dic.get(12, None))
    
open_space = []
for dic in non_zero:
    open_space.append(dic.get(21, None))

low_intensity = []
for dic in non_zero:
    low_intensity.append(dic.get(22, None))
    
medium_intesnity = []
for dic in non_zero:
    medium_intesnity.append(dic.get(23, None))

high_intensity = []
for dic in non_zero:
    high_intensity.append(dic.get(24, None))
    
barren_land = []
for dic in non_zero:
    barren_land.append(dic.get(31, None))
    
deciduous_forest = []
for dic in non_zero:
    deciduous_forest.append(dic.get(41, None))

evergreen_forest = []
for dic in non_zero:
    evergreen_forest.append(dic.get(42, None))
    
mixed_forest = []
for dic in non_zero:
    mixed_forest.append(dic.get(43, None))

dwarf_scrub = []
for dic in non_zero:
    dwarf_scrub.append(dic.get(51, None))
    
shrub_scrub = []
for dic in non_zero:
    shrub_scrub.append(dic.get(52, None))
    
grassland = []
for dic in non_zero:
    grassland.append(dic.get(71, None))

sedge = []
for dic in non_zero:
    sedge.append(dic.get(72, None))
    
lichens = []
for dic in non_zero:
    lichens.append(dic.get(73, None))

moss = []
for dic in non_zero:
    moss.append(dic.get(74, None))
    
pasture_hay = []
for dic in non_zero:
    pasture_hay.append(dic.get(81, None))

cultivated_crops = []
for dic in non_zero:
    cultivated_crops.append(dic.get(82, None))
    
woody_wetlands = []
for dic in non_zero:
    woody_wetlands.append(dic.get(90, None))

emergent_wetlands = []
for dic in non_zero:
    emergent_wetlands.append(dic.get(95, None))
    


In [122]:
emergent_wetlands = []
for dic in non_zero:
    emergent_wetlands.append(dic.get(95, None))

[23718,
 101674,
 134001,
 18876,
 34084,
 20636,
 84483,
 103966,
 67998,
 35487,
 100143,
 36187,
 96784,
 56462,
 41352,
 58400,
 64027,
 28472,
 116464,
 134510,
 8547,
 60050,
 240413,
 94291,
 44329,
 59692,
 48336,
 7130,
 59338,
 48120,
 61242,
 135220,
 68649,
 44344,
 136268,
 79750,
 93413,
 48635,
 48282,
 87074,
 42757,
 84534,
 49117,
 118301,
 70495,
 7854,
 55020,
 91636,
 53706,
 51622,
 46348,
 43366,
 100179,
 99956,
 178999,
 79933,
 154586,
 41312,
 81515,
 68085,
 344133,
 112077,
 103642,
 51067,
 50168,
 84087,
 69733,
 51601,
 131663,
 160920,
 56110,
 40440,
 38445,
 139671,
 23717,
 36342,
 29979,
 34734,
 173093,
 32972,
 85365,
 112576,
 11168,
 73291,
 41936,
 15509,
 128194,
 54624,
 56117,
 135545,
 75742,
 47622,
 41087,
 52436,
 135191,
 54906,
 120172,
 50429,
 95180,
 26587,
 71782,
 150133,
 88057,
 41498,
 27739,
 44994,
 45351,
 63762,
 27296,
 39114,
 36870,
 176921,
 5231,
 23539,
 56486,
 57728,
 66115,
 69677,
 83587,
 43866,
 57237,
 114775,


In [17]:
land_cover_2004 = pd.DataFrame.from_dict(non_zero)
land_cover_2004 = land_cover_2004.fillna(0)
land_cover_2004 = land_cover_2004.rename(columns={11: "open_water", 12:"ice_snow", 21:"open_space",22:"low_intensity",23:"medium_intensity",24:"high_intensity",31:"barren_land",41:"deciduous_forest",42:"evergreen_forest",43:"mixed_forest",51:"dwarf_scrub",52:"shrub_scrub",71:"grassland",72:"sedge",73:'lichens',74:"moss",81:"pasture_hay",82:"cultivated_crops",90:"woody_wetlands",95:"emergent_wetlands"})

land_cover_2004

,COUNTYFP,NAME,NAMELSAD,STATEFP,open_water,open_space,low_intensity,medium_intensity,high_intensity,barren_land,...,woody_wetlands,emergent_wetlands,count,majority,minority,unique,ice_snow,0,255,2
0,069,Wahkiakum,Wahkiakum County,53,63231.0,41169.0,7681.0,1076.0,149.0,1783.0,...,19759.0,51163.0,825055,42.0,24.0,15,0.0,0.0,0.0,0.0
1,011,De Baca,De Baca County,35,7182.0,23695.0,3432.0,726.0,181.0,608.0,...,5195.0,17296.0,6717692,52.0,24.0,13,0.0,0.0,0.0,0.0
2,109,Lancaster,Lancaster County,31,31525.0,94859.0,121238.0,54783.0,24794.0,1149.0,...,7684.0,12332.0,2435666,82.0,42.0,15,0.0,0.0,0.0,0.0
3,129,Nuckolls,Nuckolls County,31,8031.0,50970.0,9410.0,1287.0,448.0,222.0,...,6623.0,2096.0,1657070,82.0,52.0,14,0.0,0.0,0.0,0.0
4,099,Minnehaha,Minnehaha County,46,35197.0,132527.0,53858.0,30078.0,15066.0,5384.0,...,1210.0,40962.0,2342066,82.0,42.0,15,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,233,Webster,Webster County,21,7167.0,41477.0,5707.0,1916.0,828.0,2331.0,...,33926.0,5475.0,965939,82.0,24.0,15,0.0,0.0,0.0,0.0
2467,047,Garfield,Garfield County,40,13279.0,127951.0,40280.0,17225.0,7739.0,245.0,...,635.0,990.0,3050685,82.0,31.0,15,0.0,0.0,0.0,0.0
2468,123,Gilmer,Gilmer County,13,13408.0,87678.0,9130.0,2374.0,690.0,1031.0,...,550.0,158.0,1240172,41.0,95.0,15,0.0,0.0,0.0,0.0
2469,089,Madison,Madison County,28,78250.0,96895.0,41397.0,13889.0,2768.0,783.0,...,285189.0,23044.0,2135411,81.0,31.0,15,0.0,0.0,0.0,0.0


In [16]:
land_cover_2004.describe()

,open_water,open_space,low_intensity,medium_intensity,high_intensity,barren_land,deciduous_forest,evergreen_forest,mixed_forest,shrub_scrub,...,cultivated_crops,woody_wetlands,emergent_wetlands,count,majority,minority,unique,ice_snow,0,255
count,2.120000e+03,2120.000000,2120.000000,2120.000000,2120.000000,2.120000e+03,2.120000e+03,2.120000e+03,2.120000e+03,2.120000e+03,...,2.120000e+03,2.120000e+03,2.120000e+03,2.120000e+03,2120.000000,2120.000000,2120.000000,2120.000000,2.120000e+03,2.120000e+03
mean,1.539664e+05,79921.326887,40020.266509,17219.894811,6083.576415,2.362884e+04,2.559743e+05,3.329249e+05,9.984112e+04,6.127767e+05,...,4.542934e+05,1.205687e+05,4.335533e+04,2.900982e+06,64.848113,51.991038,14.606132,116.384434,4.887408e+03,6.841460e+04
std,6.188734e+05,66810.822130,59580.690638,43975.156952,17335.053711,1.741586e+05,3.521269e+05,9.726070e+05,2.161113e+05,2.529437e+06,...,6.333567e+05,2.822412e+05,1.415693e+05,3.832096e+06,39.579486,44.327074,2.413596,1485.395665,1.052541e+05,4.742720e+05
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,5.888000e+03,11.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000e+00
25%,8.074000e+03,42858.500000,10449.750000,2018.000000,456.000000,7.810000e+02,1.155275e+04,7.477500e+02,2.091750e+03,1.424500e+03,...,9.778500e+03,4.107500e+03,1.200000e+03,1.278740e+06,41.000000,24.000000,15.000000,0.000000,0.000000e+00,0.000000e+00
50%,2.133500e+04,63504.000000,21191.000000,5043.000000,1379.500000,2.112500e+03,1.096825e+05,1.785050e+04,2.810900e+04,1.266900e+04,...,1.707290e+05,2.314750e+04,7.066500e+03,1.864068e+06,52.000000,42.000000,15.000000,0.000000,0.000000e+00,0.000000e+00
75%,6.262900e+04,94375.500000,42811.750000,12916.250000,4154.000000,5.693000e+03,3.656520e+05,2.413265e+05,1.154462e+05,1.034318e+05,...,7.155070e+05,1.105030e+05,2.865125e+04,2.808304e+06,82.000000,81.000000,15.000000,0.000000,0.000000e+00,0.000000e+00
max,1.560754e+07,822876.000000,922643.000000,727355.000000,392784.000000,4.368347e+06,2.519542e+06,1.311405e+07,4.091182e+06,4.230925e+07,...,5.924337e+06,7.323837e+06,3.559641e+06,5.370303e+07,255.000000,255.000000,17.000000,41349.000000,4.468231e+06,8.291176e+06


In [18]:
low_intensity = land_cover_2004['low_intensity']
low_intensity = list(low_intensity)

open_space = land_cover_2004['open_space']
open_space = list(open_space)

medium_intensity = land_cover_2004['medium_intensity']
medium_intensity = list(medium_intensity)

high_intensity = land_cover_2004['high_intensity']
high_intensity = list(high_intensity)


count = land_cover_2004['count']
count = list(count)

In [19]:
deciduous_forest = land_cover_2004['deciduous_forest']
deciduous_forest = list(deciduous_forest)

evergreen_forest = land_cover_2004['evergreen_forest']
evergreen_forest = list(evergreen_forest)

mixed_forest = land_cover_2004['mixed_forest']
mixed_forest = list(mixed_forest)


In [20]:
pasture_hay = land_cover_2004['pasture_hay']
pasture_hay = list(pasture_hay)

cultivated_crops = land_cover_2004['cultivated_crops']
cultivated_crops = list(cultivated_crops)

In [21]:
#calculate agriculture
from operator import truediv

ag =  [sum(x) for x in zip(cultivated_crops, pasture_hay)]
pctAg =  map(truediv, ag, count)
pctAg = [x for x in pctAg]
pctAg

[0.028291447236850877,
 0.006707809765615929,
 0.5525954708075738,
 0.5793521094461912,
 0.8190875065006707,
 0.00557285652994746,
 0.0014374105230007352,
 0.3551525342301225,
 0.6623282258608376,
 0.16360944124244567,
 0.7280051656589589,
 0.1482611414463915,
 0.16122541772936475,
 0.09772985348830768,
 0.617818645791271,
 0.045533150118087354,
 0.1723666532526269,
 0.08323444613681041,
 0.32610263269092093,
 0.03222563373164244,
 0.03325968688380491,
 0.22428411511289725,
 0.1403419709386273,
 0.0006059639309880704,
 1.3990611246796639e-05,
 0.7996873873490878,
 0.23627257954936634,
 0.04827316743452086,
 0.10357812864161428,
 0.0687556033361924,
 0.39238679503261537,
 0.31172235973240003,
 0.1890326602440316,
 0.09808976942241457,
 0.8169884757798254,
 0.2494885728283512,
 0.14200473721460205,
 0.8871086993270645,
 0.41826041354953974,
 0.1919308242669764,
 0.002031968089718762,
 0.38216401974233144,
 0.45386819018913027,
 0.1166050369100296,
 0.06350190113912534,
 0.201572091641229

In [22]:
#calculate forest

forest =  [sum(x) for x in zip(deciduous_forest, evergreen_forest, mixed_forest)]
forest =  map(truediv, forest, count)
pctForest = [x for x in forest]
pctForest

[0.5482713273660544,
 3.498225283326476e-05,
 0.04921077027802662,
 0.02655047765031049,
 0.013323279531832152,
 0.008672011781950723,
 0.6370480644075485,
 0.5344618757464316,
 0.00011689373909524938,
 0.0722166598663539,
 0.08217565802791062,
 0.7736773371229158,
 0.5978794820163703,
 0.5457768258569595,
 0.17982964561143616,
 0.37239972987936093,
 0.6365200418988595,
 0.13033950879877484,
 0.6012867910400146,
 0.08193797393247199,
 0.11840448530634547,
 0.6340243273208346,
 0.4922046658521627,
 0.17164547103837446,
 0.4891229015023359,
 0.11702475218983181,
 0.005302225507954684,
 0.8872219149657592,
 0.05275041159479096,
 0.4114779611812512,
 0.465639758540186,
 0.5876225000365053,
 0.23415388137367202,
 0.6091360040497483,
 0.08970111800657855,
 0.539734244864224,
 0.12230327146144078,
 0.049410647783718065,
 0.49503185518556597,
 0.0767277729425884,
 0.023293927219123045,
 0.0010515415643330677,
 0.1560008184927391,
 0.7390144845753787,
 0.08040713448944789,
 0.6817325448951161,


In [23]:
#calulate suburban

suburban =  [sum(x) for x in zip(low_intensity, medium_intensity, open_space)]
pctSub =  map(truediv, suburban, count)
pctSub = [x for x in pctSub]
pctSub

[0.0605123294810649,
 0.004146215694318823,
 0.1112139349155426,
 0.03721448098149143,
 0.09242395389369898,
 0.008486763406629043,
 0.01464641868084719,
 0.059273005315099825,
 0.04631758783890106,
 0.0154902632576629,
 0.15781139366028105,
 0.03843849595532762,
 0.1923718951542533,
 0.037979383939003214,
 0.042312913614258485,
 0.0056707358310248744,
 0.06731462524097481,
 0.008683308715485703,
 0.04051182718549221,
 0.10521868674394375,
 0.7579407502435084,
 0.12166159921872628,
 0.24968429922879673,
 0.006700473958600435,
 0.0029380785073873186,
 0.04164655086565401,
 0.02698794759084734,
 0.03506831362689997,
 0.024571843400123035,
 0.1352271801101456,
 0.04430104429520535,
 0.06386967582549624,
 0.08483461536244463,
 0.13775348149324546,
 0.05980425290643333,
 0.07002718885219661,
 0.1653172532944883,
 0.055104389534238435,
 0.05448811961076529,
 0.028100908107853897,
 0.2629158597566263,
 0.024428385336871992,
 0.03532283367383064,
 0.05241400546393235,
 0.020424995725358486,
 0

In [24]:
#multi-family urban


urban =  [sum(x) for x in zip(high_intensity)]
pctUrban =  map(truediv, urban, count)
pctUrban = [x for x in pctUrban]
pctUrban

[0.0001805940210046603,
 2.694377771413158e-05,
 0.010179556638718116,
 0.0002703567139589758,
 0.006432781996749878,
 3.928343925740435e-05,
 7.187052615003676e-05,
 0.0011249336035429487,
 0.0009745756117467832,
 2.433662727048374e-05,
 0.009834225736221707,
 0.00019096120751326896,
 0.014388781977788404,
 0.0002718308371056273,
 0.0007345234067651124,
 1.1399345261067476e-05,
 0.0008161160619973717,
 2.3988807781841303e-05,
 0.000493576879414353,
 0.0035774715463080466,
 0.08889839165656997,
 0.004740131866719275,
 0.01557618107099223,
 5.920156438954063e-05,
 1.053056760511575e-06,
 0.0004757908640001025,
 0.001585923006853489,
 0.00010515181154434068,
 0.0002601347535363235,
 0.0026779580763722595,
 0.0006281731013455847,
 0.0008063340534571263,
 0.0014814497620539695,
 0.003108032993538269,
 0.001844568488580388,
 0.0016062740052881757,
 0.008244804308356597,
 0.0017647269753074824,
 0.0006745339247109528,
 6.963878572869151e-05,
 0.16893950342515246,
 2.2627313996407914e-05,
 0.

In [25]:
land_cover_2004['pct_sub_2004'] = pctSub
land_cover_2004['pct_urban_2004'] = pctUrban
land_cover_2004['pct_forest_2004'] = pctForest
land_cover_2004['pct_agri_2004'] = pctAg



land_cover_2004.head(100)

,COUNTYFP,NAME,NAMELSAD,STATEFP,open_water,open_space,low_intensity,medium_intensity,high_intensity,barren_land,...,minority,unique,ice_snow,0,255,2,pct_sub_2004,pct_urban_2004,pct_forest_2004,pct_agri_2004
0,069,Wahkiakum,Wahkiakum County,53,63231.0,41169.0,7681.0,1076.0,149.0,1783.0,...,24.0,15,0.0,0.0,0.0,0.0,0.060512,0.000181,0.548271,0.028291
1,011,De Baca,De Baca County,35,7182.0,23695.0,3432.0,726.0,181.0,608.0,...,24.0,13,0.0,0.0,0.0,0.0,0.004146,0.000027,0.000035,0.006708
2,109,Lancaster,Lancaster County,31,31525.0,94859.0,121238.0,54783.0,24794.0,1149.0,...,42.0,15,0.0,0.0,0.0,0.0,0.111214,0.010180,0.049211,0.552595
3,129,Nuckolls,Nuckolls County,31,8031.0,50970.0,9410.0,1287.0,448.0,222.0,...,52.0,14,0.0,0.0,0.0,0.0,0.037214,0.000270,0.026550,0.579352
4,099,Minnehaha,Minnehaha County,46,35197.0,132527.0,53858.0,30078.0,15066.0,5384.0,...,42.0,15,0.0,0.0,0.0,0.0,0.092424,0.006433,0.013323,0.819088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,069,Huntington,Huntington County,18,19120.0,61576.0,22744.0,6276.0,2206.0,855.0,...,52.0,15,0.0,0.0,0.0,0.0,0.081195,0.001977,0.128683,0.758303
96,209,Scott,Scott County,21,2413.0,35286.0,16724.0,7680.0,3746.0,485.0,...,90.0,15,0.0,0.0,0.0,0.0,0.072674,0.004561,0.335824,0.578512
97,053,Decatur,Decatur County,19,11859.0,36901.0,25440.0,1306.0,232.0,410.0,...,24.0,15,0.0,0.0,0.0,0.0,0.041458,0.000151,0.218125,0.713788
98,169,Wabash,Wabash County,18,23193.0,54876.0,19427.0,4941.0,1855.0,2162.0,...,43.0,15,0.0,0.0,0.0,0.0,0.065412,0.001531,0.143792,0.750989


In [26]:
land_cover_2004 = land_cover_2004[['STATEFP', 'COUNTYFP', 'NAMELSAD','pct_sub_2004', 'pct_urban_2004', 'pct_forest_2004', 'pct_agri_2004','unique']]
land_cover_2004
land_cover_2004.to_csv('land_cover_2004.csv',sep = ',', index=False)
#Results.to_csv('H:/WorkingData/CountyPctAgPctUrb.csv', sep = ',', index=False)